___
# Ciência dos Dados - PROJETO 2

___
## Guilherme Leite

## Leonardo Neves

___

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Usuario\Desktop\Insper\2018\DPCD\DPCD\P2


In [3]:
df = pd.read_excel('spamham.xlsx') #le o arquivo

df = df.replace(',','',regex=True)
df = df.replace('@','',regex=True)
df = df.replace(';','',regex=True)
df = df.replace('  ','',regex=True)
df = df.replace(':','',regex=True)
df = df.replace('"','',regex=True)
df = df.replace('\\*','',regex=True)
df = df.replace('\*','',regex=True)
df = df.replace('_','',regex=True)
df = df.replace('\$','',regex=True)
df = df.replace('rt','',regex=True)
df = df.replace('//','',regex=True)
df = df.replace('/','',regex=True)
df = df.replace('#','',regex=True)
df = df.replace('\(','',regex=True)
df = df.replace('\)','',regex=True)
df = df.replace('\'','',regex=True)
df = df.replace('\?','',regex=True)
df = df.replace('\-','',regex=True)
df = df.replace('!','',regex=True)

df = df.sample(frac=1) #Deixa o df em uma ordem aleatoria

L = len(df)
Learn = int(L*0.75) #tamanho do df de treinamento
Test = int(L*0.25) # tamenho do df de teste

dfLearn = df.iloc[0:Learn] #df de treinamento
dfLearn = dfLearn.reset_index(drop=True)

dfTest = df.iloc[Learn:L] #df de teste
dfTest = dfTest.reset_index(drop=True)


In [4]:
Ham = []
Spam =[]
NSpam = 0

for index, row in dfLearn.iterrows(): #Percorre o df linha por linha
    if row['Class'] == 'spam':
        linha = row['Email']
        Linha = linha.split() #Separa as palavras do conteúdo do email
        NSpam +=1 #Adiciona +1 ao numero de emails de Spam
        for palavra in Linha:
            Hm = []
            for letra in palavra:  #Separa as letras das palavras
                if letra.isalpha() == True: # Se a letrea é alfabética:
                    Hm.append(letra) 
            hm  = ''.join(Hm) #Junta as letras em palavra
            if len(hm)>1: # Filtra palavras com menos de duas Letras
                Spam.append(hm)
    else:
        linha = str(row['Email'])
        Linha = linha.split()  #Separa as palavras do conteúdo do email
        for palavra in Linha:
            Hm = []
            for letra in palavra:
                if letra.isalpha() == True: # Se a letrea é alfabética:
                    Hm.append(letra)
            hm  = ''.join(Hm) #Junta as letras em palavra
            if len(hm)>1: # Filtra palavras com menos de duas Letras
                Ham.append(hm)

___
## 3. Classificador Naive-Bayes

In [5]:
P_Spam = NSpam/len(dfLearn) #Probabilidade de uma mensagem ser SPAM 
P_Ham = 1-P_Spam #Probabilidade de uma mensagem ser HAM
print('Probabilidade de uma mensagem ser SPAM: ',P_Spam, 
      '\nProbabilidade de uma mensagem ser HAM: ',P_Ham)

Probabilidade de uma mensagem ser SPAM:  0.134688995215311 
Probabilidade de uma mensagem ser HAM:  0.865311004784689


In [6]:
vs= pd.Series(Spam)
vh= pd.Series(Ham)

In [15]:
lsp= len(Spam)
lha= len(Ham)
x = "Thanks for your subscription to Ringtone UK your mobile will be charged å£5/month Please confirm by replying YES or NO. If you reply NO you will not 0be charge"
def email(x): #Classificador
    sep=str(x).split(' ') #Separa o conteudo do email por palavras
    spams = []
    hams = []
    for i in range(len(sep)):
        spams.append((Spam.count(sep[i])/lsp)) #adicona a lista da palavra ser Spam
        hams.append((Ham.count(sep[i])/lha))  #adicona a lista da palavra ser Ham
    pps = np.sum(spams) #soma as probabilidades
    pph = np.sum(hams)
    if pph > pps:  #compara as probabilidades dando uma maior ponderação para o ham
        return "Ham"
    else:
        return "Spam"
email(x)#Teste

'Spam'

___
## 5. Qualidade do Classificador alterando a base de treinamento

In [16]:
for index, row in dfLearn.iterrows(): #Percorre o df linha por linha
    if row['Class'] == 'spam':
        linha = row['Email']
        Linha = linha.split() #Separa as palavras do conteúdo do email
        for palavra in Linha:
            Hm = []
            for letra in palavra:  #Separa as letras das palavras
                if letra.isalpha() == True: # Se a letrea é alfabética:
                    Hm.append(letra) 
            hm  = ''.join(Hm) #Junta as letras em palavra
            if len(hm)>1: # Filtra palavras com menos de duas Letras
                row['Email'] = hm
    else:
        linha = str(row['Email'])
        Linha = linha.split()  #Separa as palavras do conteúdo do email
        for palavra in Linha:
            Hm = []
            for letra in palavra:
                if letra.isalpha() == True: # Se a letrea é alfabética:
                    Hm.append(letra)
            hm  = ''.join(Hm) #Junta as letras em palavra
            if len(hm)>1: # Filtra palavras com menos de duas Letras
                row['Email'] = hm

In [17]:
falso_positivo = []
positivo_verdadeiro = []
falso_negativo = []
negativo_verdadeiro = []

H = 0
S = 0
for u in range(len(dfTest)):
    x = dfTest.Email[u]
    if dfTest.Class[u] == 'ham':
        H+=1
        Classifica = email(x)
        if Classifica == 'Ham':
            positivo_verdadeiro.append("pv")
        if Classifica == 'Spam' :
            falso_positivo.append("fp")
            
    elif dfTest.Class[u] == 'spam':
        S+=1
        if Classifica == 'Spam':
            negativo_verdadeiro.append("nv")
        if Classifica == 'Ham' :
            falso_negativo.append("fn")


In [18]:
FP = len(falso_positivo)/len(dfTest)*100  #Porcentagens sobre o total
PV = len(positivo_verdadeiro)/len(dfTest)*100  #Porcentagens sobre o total
FN = len(falso_negativo)/len(dfTest)*100  #Porcentagens sobre o total
NV = len(negativo_verdadeiro)/len(dfTest)*100  #Porcentagens sobre o total

FPP = len(falso_positivo)/H*100         #Porcentagens sobre Positivos/Negativos
FNN = len(positivo_verdadeiro)/S*100
VPP = len(falso_negativo)/H*100
VNN = len(negativo_verdadeiro)/S*100
print("FP",FP)
print("FN",FN)
print("VP",PV)
print("VN",NV,"\n")

print("FP/P",FPP)
print("FN/N",FNN)
print("VP/P",VPP)
print("VN/N",VNN)

FP 23.242467718794835
FN 8.464849354375897
VP 63.55810616929699
VN 4.734576757532281 

FP/P 26.776859504132233
FN/N 481.52173913043475
VP/P 9.75206611570248
VN/N 35.869565217391305


Como é possível perceber pelos valores apontados acima, o classificador funcionou para a maioria dos casos, 62,63%, porem cerca de 24,68% das mensagens foram falsas positivas e 9,89% foram falsas negativas, no caso do filtro de spam, é ideal ocorrerem o menor número de falsas negativas uma vez que essas seriam mensagens poderiam ser importantes para o usuário, por essa motivo.